import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.random.set_seed(42)
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

device = 'GPU' if 'GPU' in tf.test.gpu_device_name() else 'CPU/TPU'
print('Device:', device)

import os, gc, random, datetime
if device == 'GPU':
    import cudf
    import cupy as cp
import pandas as pd
import numpy as np
import janestreet
import xgboost as xgb
import datatable as dtable
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load
from time import time
from numba import njit

print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

PyTorch vs. Keras

PyTorch: Mathmeticians, experienced researchers; faster than Keras and has better debugging capabilities, outstanding community support and active development.

Keras: developers; a plug-and-play framework that lets them build, train, and evaluate their models quickly. Keras also offers more deployment options and easier model export, offers more deployment options and easier model export, excellent access to reusable code and tutorials, small datasets, rapid prototyping, and multiple back-end support.  It runs on Linux, MacOS, and Windows. 



|                                  |                     **Keras**                    |            **PyTorch**           |          **TensorFlow**          |
|:--------------------------------:|:------------------------------------------------:|:--------------------------------:|:--------------------------------:|
|           **API Level**          |                       High                       |                Low               |           High and Low           |
|         **Architecture**         |             Simple, concise, readable            |      Complex, less readable      |          Not easy to use         |
|           **Datasets**           |                 Smaller datasets                 | Large datasets, high performance | Large datasets, high performance |
|           **Debugging**          | Simple network, so debugging is not often needed |    Good debugging capabilities   |  Difficult to conduct debugging  |
| **Does It Have Trained Models?** |                        Yes                       |                Yes               |                Yes               |
|          **Popularity**          |                   Most popular                   |        Third most popular        |        Second most popular       |
|             **Speed**            |               Slow, low performance              |      Fast, high-performance      |      Fast, high-performance      |
|          **Written In**          |                      Python                      |                Lua               |         C++, CUDA, Python        |



Reference: https://www.simplilearn.com/keras-vs-tensorflow-vs-pytorch-article#:~:text=TensorFlow%20offers%20better%20visualization%2C%20which,to%20the%20TensorFlow%20Serving%20framework.

In [3]:
import pandas as pd
